In [145]:
import matrix as m

class BlockInterface:
    def __init__(self, is_collectable, is_movable, is_walkable, name, gameBoard):
        #document the block
        """
        Parameters
        ----------
        is_collectable : bool
            Whether the block can be collected by the player for example keys or diamonds.
        is_movable : bool
            Whether the block can be moved by the player or not.
        is_walkable : bool
            Whether the player can walk on the block or not.
        """
        
        self._is_collectable = is_collectable
        self._is_movable = is_movable
        self._is_walkable = is_walkable 
        self._name = name
        self._gameBoard = gameBoard

    def walkOver(self):
        pass

    def unwalkOver(self):
        pass

    def isReachable(self):
        return False

    def set_is_collectable(self, is_collectable):
        self._is_collectable = is_collectable
    
    def get_is_collectable(self):
        return self._is_collectable

    is_collectable = property(get_is_collectable, set_is_collectable)
    
    def set_is_movable(self, is_movable):
        self._is_movable = is_movable

    def get_is_movable(self):
        return self._is_movable

    is_movable = property(get_is_movable, set_is_movable)

    def set_is_walkable(self, is_walkable):
        self._is_walkable = is_walkable

    def get_is_walkable(self):
        return self._is_walkable

    is_walkable = property(get_is_walkable, set_is_walkable)

    def get_gameBoard(self):
        return self._gameBoard

    def set_gameBoard(self, gameBoard):
        self._gameBoard = gameBoard
    
    gameBoard = property(get_gameBoard, set_gameBoard)

    def get_name(self):
        return self._name

    def set_name(self, name):
        self._name = name

    name = property(get_name, set_name)
    
    

    def __str__(self):
        return self.name[0:1]

class Wall(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, False, "Wall", gameBoard)

class Floor(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, True, "Floor", gameBoard)

class Destination(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, True, "Destination", gameBoard)

    def isReachable(self):
        #if there is no more diamonds to collect, the destination is reachable
        return super().gameBoard.remainingDiamonds == 0

class Key(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(True, False, True, "Key", gameBoard)

    def walkOver(self):
        super().get_gameBoard().has_key = True
        super().set_is_collectable(False)

    def unwalkOver(self):
        super().get_gameBoard().has_key = False
        super().set_is_collectable(True)

    def isReachable(self):
        return super().is_collectable and super().gameBoard.has_key == False

class Diamond(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(True, False, True, "Diamond", gameBoard)
    
    def isReachable(self):
        return True

class KeyDoor(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, False, "KeyDoor", gameBoard)
    
    def isReachable(self):
        return super().gameBoard.has_key == True

class Spikes(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, False, "Spikes", gameBoard)

    def walkOver(self):
        pass

class Rock(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, True, True, "Rock", gameBoard)

    def walkOver(self):
        pass

    def isReachable(self):
        return True

class Magma(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, False, "Magma", gameBoard)

    def walkOver(self):
        pass

class Hole(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, False, "Hole", gameBoard)

    def walkOver(self):
        pass

class Button(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, True, "Button", gameBoard)

    def walkOver(self):
        pass

class DoorButton(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, True, "DoorButton", gameBoard)

    def walkOver(self):
        pass

class Character(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, True, "Character", gameBoard)

    def walkOver(self):
        pass



class BlockGenerator:
    """
    Class to generate all the objects based on a fixed map
    """
    def __init__(self):
        self.map = {
            'R' : Rock,
            'D' : Diamond,
            'P' : Character,
            'M' : Wall,
            'W' : Magma,
            'A' : Destination,
            'C' : Floor,
            'K' : KeyDoor,
            'L' : Key,
            'S' : Spikes,
            'H' : Hole,
            'B' : Button,
            'U' : DoorButton,
        }

    def generate(self, target, gameBoard):
        """
        returns a new Object based on the given char
        for example:
        'R' -> Rock
        'D' -> Diamond
        """
        return  self.map[target](gameBoard)


class GameBoard:
    def __init__(self):
        self.board = []
        self.has_key = False
        self.remainingDiamonds = 0
        self.characterLocation = (0,0)
        self.destinationLocation = (0,0)

    def initialize(self, gameMap, gameBoard):
        self.board = self.generateBoard(gameMap, gameBoard)
        self.generateNumberOfCollectables()

    def generateBoard(self, gameMap, gameBoard):
        """
        Generates the board based on the given map
        """
        board = []
        generator = BlockGenerator()
        for row in gameMap:
            board.append([generator.generate(target, gameBoard) for target in row])
        return board

    def generateNumberOfCollectables(self):
        """
        Iterates over the board and counts the number of diamonds and keys
        """
        for row in self.board:
            for block in row:
                if block.name == "Diamond":
                    self.remainingDiamonds += 1
                if block.name == "Character":
                    self.characterLocation = (self.board.index(row), row.index(block))
                if block.name == "Destination":
                    self.destinationLocation = (self.board.index(row), row.index(block))

    def generateVisitedMatrix(self):
        """
        Generates a matrix of the same size as the board, but filled with False
        """
        return [[False for x in range(len(self.board[0]))] for y in range(len(self.board))]

    
    def getShortestSolutionPath(self):
        """
        Returns the shortest path to the solution
        This path should collect all the diamonds 
        """
        shortestInstructions = [] #array composed of the instructions to solve the game in the form of (L, R, U, D)
        self.getShortestSolutionPathRecursive([], shortestInstructions)
        return shortestInstructions
        

    def getShortestSolutionPathRecursive(self, instructions, shortestInstructions):
        """
        Recursive function to find the shortest path to the solution
        This path should collect all the diamonds 
        """
        if self.isSolved(): #chack if the map is solved
            if self.getInstructionsLenght(shortestInstructions) == 0:
                shortestInstructions = instructions
            elif self.getInstructionsLenght(instructions) < self.getInstructionsLenght(shortestInstructions):
                shortestInstructions = instructions 

        specialBlocksPath = self.getSpecialBlocksPath()
        
        for block in specialBlocksPath:
            moves = self.moveTo(block)
            instructions.append(moves)
            self.getShortestSolutionPathRecursive(instructions)
            instructions.pop()
            self.moveBack(moves)

    def moveBack(self, block):
        pass

    def moveTo(self, block):
        """
        Moves the character to the given block

        parameters:
        block: the block to move to, its compossed by an array of points of the map where the character moves
        for example: [(3, 3), (2, 3), (1, 3), (0, 3)] it means that the character moves from (3, 3) to (0, 3)
        incluiding the block (0, 3) in the path
        """
        list_movements = []
        for i in range(len(block) - 1):
            start_point = block[i]
            end_point = block[i+1]
            moves = self.moveCharacter(start_point, end_point)
            list_movements.append(moves)
        return list_movements
        
    def moveCharacter(self, start_point, end_point):
        """
        Moves the character to the given point
        """
        #get the block at the point and call the walkOver function
        self.board[start_point[0]][start_point[1]].walkOver()
        #update the character location
        self.characterLocation = end_point
        #return the direction of the move
        return self.getDirectionOfMovement(start_point, end_point)

    def getDirectionOfMovement(self, start_point, end_point):
        """
        Moves the character from the start_point to the end_point and returns the direction of the move
        (L, R, U, D)
        """
        direction = ""
        if start_point[0] == end_point[0]:
            if start_point[1] > end_point[1]:
                direction = "L"
            else:
                direction = "R"
        else:
            if start_point[0] > end_point[0]:
                direction = "U"
            else:
                direction = "D"

        return direction
        

    def isSolved(self):
        """
        Returns True if the map is solved
        """
        if self.remainingDiamonds != 0:
            return False
        
        if self.characterLocation != self.destinationLocation:
            return False

        return True
            
    def getInstructionsLenght(self, instructions):
        """
        Returns the lenght of the instructions
        """
        lenght = 0
        for instruction in instructions:
            lenght += len(instruction)
        return lenght
    
    
    def getSpecialBlocksPath(self):
        """
        starting from the character location, get all the speacial blocks for example diamonds, keys, etc
        """

        #list of reachable blocks each element is dict with keys: block, location where location is a list of charters, each character is a location in the board
        reachableBlocks = {} 
        #matrix to keep track of visited blocks
        visited = self.generateVisitedMatrix()
        #seacht the shortest path to each reachable location
        self.getSpecialBlocksRecursive(self.characterLocation, visited, reachableBlocks, [self.characterLocation])

        #convert the reachable blocks to a list of blocks
        reachableBlocksList = []
        for key in reachableBlocks:
            reachableBlocksList.append({"block":key,"path":reachableBlocks[key]})
        
        return reachableBlocksList


    def getSpecialBlocksRecursive(self, location, visited, reachableBlocks, path):
        """
        Recursive function to get the shortest path to all the reachable blocks
        """
        curr_row, curr_col = location

        #if the current block is already visited, return
        if visited[curr_row][curr_col]:
            return
        
        #mark the current block as visited
        visited[curr_row][curr_col] = True

        #get the current block
        block = self.board[curr_row][curr_col]

        #if the current block is not walkable, return
        if not block.is_walkable:
            return
        
        #if the current block is a special block, add it to the reachable blocks
        if block.isReachable():
            if block not in reachableBlocks:
                reachableBlocks[block] = path.copy()
            else:
                if len(path) < len(reachableBlocks[block]):
                    reachableBlocks[block] = path.copy()
            return

        #get the next blocks
        next_blocks = self.getNeighbors(location)

        #if there are no next blocks, return
        if not next_blocks:
            return

        #for each next block, call the function recursively
        for next_block in next_blocks:
            path.append(next_block)
            self.getSpecialBlocksRecursive(next_block, visited, reachableBlocks, path)
            path.pop()

        
    def getNeighbors(self, location):
        """
        returns a list of neighbors of the given location
        """
        neighbors = []
        curr_row, curr_col = location
        for next_row, next_col in [[curr_row + 1, curr_col], [curr_row - 1, curr_col], [curr_row, curr_col + 1], [curr_row, curr_col - 1]]:
            if next_row >= 0 and next_row < len(self.board) and next_col >= 0 and next_col < len(self.board[0]):
                neighbors.append((next_row, next_col))

        return neighbors
    
    def __str__(self) -> str:
        map_str = ""
        for row in self.board:
            for block in row:
                map_str += str(block)
            map_str += "\n"
        return map_str

In [146]:
# test for generator class
gameBoard = GameBoard()
generator = BlockGenerator()
print(isinstance(generator.generate("L", gameBoard), Key) == True)
print(isinstance(generator.generate("C", gameBoard), Floor) == True)
print(isinstance(generator.generate("A", gameBoard), Destination) == True)




True
True
True


In [147]:
#test walk over and undwalkover
gameBoard = GameBoard()

#TEST 1 for exit block
keyBlock = BlockGenerator().generate('L', gameBoard)

print(keyBlock.isReachable() == 1)
keyBlock.walkOver()
print(keyBlock.isReachable() == 0)



True
True


In [148]:
# test for direction of movement function
board = GameBoard()
print( board.getDirectionOfMovement((0, 0), (0, 1)) == "R")
print( board.getDirectionOfMovement((0, 0), (0, -1)) == "L")
print( board.getDirectionOfMovement((0, 0), (1, 0)) == "D")
print( board.getDirectionOfMovement((0, 0), (-1, 0)) == "U")

True
True
True
True


In [149]:
#test for is solved function
mapReachable = [
    ['C', 'C',],
    ['A', 'P',],
]

board = GameBoard()
board.initialize(mapReachable, board)

print(board.destinationLocation == (1, 0))
print(board.characterLocation == (1, 1))
print(board.remainingDiamonds == 0)
print(board.isSolved() == False)
board.characterLocation = (1, 0)
print(board.isSolved() == True)
board.remainingDiamonds = 1
print(board.isSolved() == False)



[[<__main__.Floor object at 0x000002B62BAA62E0>, <__main__.Floor object at 0x000002B62BAA6250>], [<__main__.Destination object at 0x000002B62B43D5E0>, <__main__.Character object at 0x000002B62B43D370>]]
True
True
True
True
True
True


In [150]:
#test of getting the reachable blocks
mapReachable = [
    ['L', 'C', 'C', 'D'],
    ['M', 'M', 'C', 'C'],
    ['C', 'M', 'C', 'C'],
    ['C', 'C', 'C', 'P'],
]

board = GameBoard()
board.initialize(mapReachable, board)

print(board.characterLocation == (3,3))
print(board.remainingDiamonds == 1)
print(board.getSpecialBlocksPath())

[[<__main__.Key object at 0x000002B62B43D8E0>, <__main__.Floor object at 0x000002B62B43D220>, <__main__.Floor object at 0x000002B62B43D760>, <__main__.Diamond object at 0x000002B62B43D7C0>], [<__main__.Wall object at 0x000002B62B43DAF0>, <__main__.Wall object at 0x000002B62B43D040>, <__main__.Floor object at 0x000002B62B43D790>, <__main__.Floor object at 0x000002B62B230970>], [<__main__.Floor object at 0x000002B62B230CA0>, <__main__.Wall object at 0x000002B62B2307F0>, <__main__.Floor object at 0x000002B62B230820>, <__main__.Floor object at 0x000002B62B2306A0>], [<__main__.Floor object at 0x000002B62B2301F0>, <__main__.Floor object at 0x000002B62B2303A0>, <__main__.Floor object at 0x000002B62B230DF0>, <__main__.Character object at 0x000002B62BBE4580>]]
True
True
[{'block': <__main__.Diamond object at 0x000002B62B43D7C0>, 'path': [(3, 3), (2, 3), (1, 3), (0, 3)]}, {'block': <__main__.Key object at 0x000002B62B43D8E0>, 'path': [(3, 3), (2, 3), (1, 3), (1, 2), (0, 2), (0, 1), (0, 0)]}]


In [113]:
board = GameBoard()
board.initialize(m.Level1, board)

print(board.getSpecialBlocksPath())

[{'block': <__main__.Diamond object at 0x000002B62B731640>, 'path': [(5, 2), (5, 3)]}]


In [114]:
#test of generation a signle block
gameBoard = GameBoard()

#TEST 1 for exit block
exitBlock = BlockGenerator().generate('A', gameBoard)

#test to get if the exit is reachable when there are no diamonds
gameBoard.remainingDiamonds = 0
print("test 1: ", exitBlock.isReachable() == True)

#test to get if the exit is reachable when there are diamonds
gameBoard.remainingDiamonds = 1
print("test 2: ", exitBlock.isReachable() == False)

#-------------
#TEST 2 for Door Block
doorBlock = BlockGenerator().generate('K', gameBoard)

gameBoard.has_key = True
print("test 3: ", doorBlock.isReachable() == True)

gameBoard.has_key = False
print("test 4: ", doorBlock.isReachable() == False)




test 1:  True
test 2:  True
test 3:  True
test 4:  True
